### TUTORIAL PARA EL USO DE REDAPYX A NIVEL DISTRITAL
Redapyx es una libreria escrita en python que permite descargar datos censales del procesador estadístico de Redatam (https://censos2017.inei.gob.pe/redatam/) y transformar sus resultados en una matriz que usa el ubigeo como variable única de indexación.

Para el siguiente ejemplo, vamos a usar el módulo de Redapy para explorar las características de la población que vive en viviendas alquiladas en Lima metropolitana en el año 2017. Para ello, lo primero que haremos será llamar la libreria redapy y la libreria pandas, esencial para el procesamiento de bases de datos.

_OJO: Recuerda que las tablas que devuelve redapy **siempre** son dataframes de la libreria pandas_

In [1]:
import redapyx
import pandas as pd
service_path=r'C:\Users\malag\Desktop\chromedriver.exe'

### Quick start

Antes de identificar a la población que vive en viviendas alquiladas, identificamos la condición de las viviendas censadas. Para ello usaremos la variable "vivienda.C2P13" la cual recoge las respuestas sobre la tenencia de la vivienda en el módulo de vivienda del censo 2017. Como nuestra consulta utiliza solo una variable, usamos el valor "frequency" en el parámetro denominado como "tipo" (tipo="Frequency"). Para obtener el resultado esperado, debemos especificar además el nivel de salida de la variable y el ubigeo de interés, que en nuestro caso corresponde al ubigeo de la provincia de Lima Metropolitana (1501). De acuerdo a las indicaciones previas, los parámetros son:
- tipo ="Frequency"
- var1="vivienda.C2P13" 
- area_break="distrito"
- selection="1501"

In [2]:
redapyx.get(service_path=service_path, tipo='Frequency',var1="poblacion.C5P2",area_break="distrito",selection="1501",print_query=True, output="Excel",factor_exp="Poblacio.FACTORPOND")

RUNDEF Job
    SELECTION INLINE, Provinci 1501


TABLE TABLE1
    AS FREQUENCY
    AREABREAK Distrito
    OF Poblacio.C5P2
    WEIGHT Poblacio.FACTORPOND
  Scraping starts
  REDATAM webpage opened successfully
  Output table loaded successfully
  Table was scraped successfully in: 0:00:05.257159
  Table was cleaned successfully in: 0:00:00
  Excel file generated successfully


In [ ]:
redapyx.get(service_path=service_path, tipo='',var1="vivienda.C2P13",area_break="distrito",selection="1501")

El resultado nos muestra que la consulta se ejecutó en cerca de 7 segundos y devolvió una base de datos similar a la que genera una consulta desde la web de REDATAM: una base de datos con una tabla por **UBIGEO**. Este formato se puede apreciar mejor al filtral un determinado número de filas, en este caso filtraremos las filas que van desde 17 hasta la 31. Para ello usaremos el comando _iloc_ de la libreria pandas.

_Nota que hemos nombrado el resultado con el nombre de "df", que es una convención para definir a un dataframe_

In [ ]:
df=redapyx.get(service_path=service_path, tipo='Frequency',var1="vivienda.C2P13",area_break="distrito",selection="1501", pivot=True)
df.iloc[17:31]

Este tipo de estructura de datos, si bien resulta de fácil lectura, suele ser poco útil para el análisis comparado ya que no permite organizar fácilmente los datos en una matriz donde cada columna corresponda a una varible y cada fila a un caso como, por ejemplo, uno de los distritos de Lima Metropolitana. Por lo tanto, se requiere transformar esta base de datos compuesta por tablas individuales con información por UBIEGO en una matriz de datos de 2x2 que facilite su análisis y visualización. 

**Redapy** permite transformar estas tablas usando el submódulo **frequency**, el cual recoge los datos de cada tabla y agrupa las respuestas por ubigeo

In [ ]:
redapyx.frequency(df)

Esta función, también cuenta con un parámetro que pivotea directamente los datos en una matriz de 2x2, la cual toma la variable de ubigeo como identificador único (index) y las categorias de la variable seleccionada como columnas de matriz.

In [ ]:
redapyx.frequency(df, pivot=True)

De esta forma, con solo **2 líneas de código** y en menos de **5 minutos** obtenemos todas las categorias de la variable en una matriz de 2x2, donde cada fila corresponde a un distrito de Lima Metropolitana y cada columna a una de las categorías de la variable de tenencia de la vivienda. Este resultado puede ser usado para graficar las frecuencias de datos o para visualizar la distribución espacial de la variable en un programa GIS. Para guardar la matriz resultante puedes usar el módulo de pandas _to_excel()_ para grabar el resultado en un archivo de excel.

El resultado final del código para realizar **la consulta, transformar los resultados y guardarlos** sería el siguiente:

In [ ]:
df=redapyx.get(service_path=service_path,tipo='Frequency',var1="vivienda.C2P13",area_break="distrito",selection="1501")

## Guardamos el resultado en un archivo excel
df.to_excel("tenencia de la vivienda_2017_sinpivot.xlsx")

In [ ]:
df=redapyx.get(service_path=service_path,tipo='Frequency',var1="vivienda.C2P13",area_break="distrito",selection="1501", pivot=True)

## Guardamos el resultado en un archivo excel
df.to_excel("tenencia de la vivienda_2017_conpivot.xlsx")

### Exploración de los resultados

Si bien la tabla anterior ya puede ser utilizada para hacer operaciones en excel o Qgis, nos interesa realizar algunos cálculos adicionales usando la libreria pandas.
Iniciaremos identificando el porcentaje de viviendas en alquiler por distrito, para ello lo primero que haremos será calcular la suma de viviendas y luego el porcentaje de viviendas alquiladas.

In [ ]:
## creamos una variable con la suma total de casos por ubigeo
df1['viviendas_total']=df1.sum(axis=1) 
## creamos una variable con el porcentaje de la variable vivienda alquilada con respecto al total de viviendas
df1['alquiler_p']=((df1.Alquilada/df1.viviendas_total)*100).round(1)
## Ahora mostramos las 5 primeras filas con la condición de que estén ordenadas de mayor a menor
df1.sort_values("alquiler_p",ascending=False).head()

La tabla anterior muestra que en los ditritos de la Victoria (150115), Santa Anita (150137), Breña (150105), San Luis (150134) y Surquillo (150141), se encuentran el mayor porcentaje de viviendas alquiladas en el año 2017, respectivamente. Por otra parte, que la vivienda se encuentre alquilada no quiere decir que la mayoría de hogares o personas del distrito vivian en viviendas alquiladas. Por esta razón calculamos la misma información pero a nivel de persona censada. Usaremos nuevamente el módulo de redapyx, pero agregaremos una variable a la consulta: el sexo del encuestado (C5P2). Esto se logra cambiando el parámetro _tipo_ por "Crosstab" que nos permite realizar consultas para 2 variables (var1="vivienda.C2P13" y var2="poblacion.C5P2").

_Nota: Además pasamos el parámetro mensajes =False para reducir el número de msj durante la ejecución de la función y solo mantener el tiempo de extracción de datos desde REDATAM_

In [ ]:
df=redapyx.get(service_path=service_path,tipo='Crosstab',var1="vivienda.C2P13", var2="poblacion.C5P2",area_break="distrito",selection="1501")
df.head(10)

In [ ]:
df=redapyx.get(service_path=service_path,tipo='Crosstab',var1="vivienda.C2P13", var2="poblacion.C5P2",area_break="distrito",selection="1501", pivot=True)
df.head(10)

In [ ]:
df=redapyx.get(service_path=service_path,var1="vivienda.C2P13", var2="poblacion.C5P2",area_break="distrito",selection="1501")
df.head(10)

### Centros poblados
Desde el 2017 el INEI pone a disposición del público la información del CPV 2017 a nivel de centros poblados y manzadas. Esta información puede ser consultada desde la web alterna de Redatam destinada este fin (https://censos2017.inei.gob.pe/bininei/RpWebStats.exe/CmdSet?BASE=CPV2017&ITEM=PROGRED&lang=esp).

De forma similar a como ocurre con la información a nivel de distrito y provincia, los datos son devueltos por el sistema en forma de tablas individuales por Ubigeo (revisar el tutorial sobre distritos acá: link). Así, a pesar del avance que representa el acceso a este nivel de desagregación para el análisis de distintos fenómenos sociales y demográficos, su uso está restringido a la capacidad que el usuario tenga para procesar la información. Asimismo, la información georreferenciada de ccpp y manzanas es distribuida a través de un WMS, formato de tipo imagen que impide la vinculación de la información con los datos censales de caracter vectorial.

A pesar de estas limitaciones, Redapyx resulta en una herramienta potente para reducir la brecha en la capacidad de análisis de los datos. A continuación, se presentan dos posibles usos de ambas bases de datos de centros poblados y manzanas haciendo uso de redapyx e información de acceso público

Ahora podemos ver que tenemos el total de hombres y mujeres por el ubigeo del distrito y el tipo de tenencia de la vivienda. A continuación, haremos algunos cambios a la tabla para lograr obtener el dato deseado.

_OJO: Si quisieramos tener una tabla multicolumna, solo sería necesario pasar el parámetro pivot=True en el submodulo cross_table: redapyx.cross_table(df2, pivot=True)_

In [ ]:
## Nos quedamos con la variable total y pivoteamos el resultado
df2=df2.reset_index().query('columna=="Total"').pivot(index="ubigeo",columns="fila",values="freq")
## sumamos todas las variables para tener el total de población por tipo de tenencia
df2['pob_total']=df2.sum(axis=1) 
## creamos una variable con el porcentaje de la variable vivienda alquilada con respecto al total de viviendas
df2['alquiler_pob_p']=((df2.Alquilada/df2.pob_total)*100).round(1)
## Ahora mostramos las 5 primeras filas con la condición de que estén ordenadas de mayor a menor
df2.sort_values("alquiler_pob_p",ascending=False).head(9)

En relación a la población en vivienda alquiladas, el distrito de Breña es el segundo distrito con más población en viviendas alquiladas, ello a pesar de ser el tercero con más viviendas con tenencia en situación de alquiler. Algo similar ocurre con el distrito de Lince (150116) el cual a pesar de ser el 7mo distrito con mayor porcentaje de viviendas en alquiler, es el cuarto con más población habitandolas.

Esta información no lleva a la siguiente pregunta, ¿cuál es el ratio entre población y viviendas alquiladas? Para resolver esta pregunta, procederemos a unir ambos _Dataframes_ (viviendas por tipo de tenencia y población por tipo de tenencia de la vivienda) usando la función _join_ de pandas y calcular nuestro indicador.

_**Este ejercicio resulta mucho más sencillo, gracias a que la función de limpieza de Redapy toma siempre al ubigeo como variable clave para indexar la información**, otra manera de hacer esta operación sería con la función de pandas **merge**_

In [ ]:
## unimos ambas BD usando la función join que utiliza los index para hacer el merge
df3=(df1[["Alquilada","viviendas_total","alquiler_p"]].
     join(df2.rename({"Alquilada":"Alquilada_pob"},axis=1)[["Alquilada_pob","pob_total","alquiler_pob_p"]]))

## Calculamos el ratio
df3["ratio"]=(df3.Alquilada_pob/df3.Alquilada).round(2)
df3.sort_values("ratio",ascending=False).head()

Luego de cruzar ambos _dataframes_, observamos que el ratio más alto de personas por vivienda alquilada se encuentra en el distrito de Villa el Salvador con casi 4 personas por vivienda alquilada, ello a pesar de que el porcentaje de población en vivienda alquiladas es de solo 12%. Por su parte, el distrito de Miraflores tiene el ratio más bajo de personas por vivienda alquilada con alrededor de 2 personas por vivienda con este tipo de tenencia, además de ser el noveno distrito con más población en viviendas alquiladas con 31% de sus vecinos alquilando una vivienda en el distrito

In [ ]:
df3.sort_values("ratio",ascending=False).tail()

### Visualización en forma de mapa
Ahora que tenemos nuestros indicadores podemos visualizarlos en forma de mapa usando los límites distritales del INEI. Para ello usaremos la libreria **_geopandas_**, la cual permite trabajar con datos vectoriales. Además, usaremos los shps de distritos 2017 descargados del siguiente enlace: https://ide.inei.gob.pe/#capas

También usaremos la librería **_matplotlib_**, la cual permite crear visualizaciones complejas

_Este mismo ejercicio de visualización se puede realizar con otros programas de GIS como, por ejemplo, QGIS_

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt ## libreria para la visualización de data

In [ ]:
## Hacemos el merge entre la BD espacial y nuestro dataframe con los indicadores que queremos graficar
dis17=gpd.read_file('Bases originales\\Límites nacionales\\Distrito.gpkg') ## en este caso nuestra BD espacial está guardada como gpkg y contiene un único layer
dis17=dis17.set_index('ubigeo').merge(df3, right_index=True, left_index=True, how='inner', validate='1:1')

## Usaremos la libreía Matplotlib para graficar el resultado
fig,ax=plt.subplots(1,2, figsize=(12,9))

dis17.plot("alquiler_pob_p",ax=ax[0], legend=True)
dis17.plot("ratio",ax=ax[1], legend=True, cmap="YlGnBu")

ax[0].set_title("Población en viviendas alquiladas\n en Lima Metropolitana %") 
ax[1].set_title("Ratio entre el total de personas en viviendas\n alquiladas y el total de viviendas alquiladas")

### Filtros para consultas en REDATAM

Ahora que conocemos el total de personas que viven en viviendas alquiladas, imaginemos que queremos conocer su permanencia en el distrito en los últimos 5 años y en qué distrito vivia antes en caso se haya mudado en los últimos 5 años. Para este ejemplo tomaremos como referencia la respuesta del jefe del hogar sobre el lugar donde vivía hace 5 años. Para realizar la consulta multivariable en vez de solicitar un reporte de 2 o más variables usaremos del parámetro _for_query_, el cual permite construir un filtro que combina  una o más variables y sus respectivos valores. El detalle de los subparametros es el siguiente:

- ***variables***: lista la variable o variables que se usarán en el filtro.
- ***category***: lista el valor de la categoría que se usará para filtrar cada variable. ___Nota que la categoría siempre es un valor númerico, para saber el valor de la respuesta puedes revisar la cédula censal___
- ***logical_exp***: lista la expresión lógica que se usará para filtrar la variable (equal, greater than, less than, etc)
- ***operator***: es el operador que se usará para filtrar las variables, puede ser "and" u "or"

Como no requerimos de más de una varible usamos la consulta del tipo "frequency" para solicitar a Redatam la información sobre el lugar donde vivía el jefe del hogar hace 5 años ("poblacion.C5P6PV"). Para garantizar que la consulta devuelva solo información de las viviendas alquiladas ("vivienda.C2P13") y del jefe del hogar ("poblacion.C5P1"), construimos nuestro filtro para ambas variables. En este caso tanto la condición de alquiler de la vivienda como la condición de jefe del hogar corresponde a los valores "1 ('category':["1", "1"]). Finalmente, especificamos en el **_operator_** que ambas condiciones se cumplan, para ello usaremos el valor ['and'] 

In [ ]:
df4=redapyx.query_2017(tipo='Frequency',var1="poblacion.C5P6PV",area_break="distrito",selection="1501",
                        for_query={'variables':["vivienda.C2P13","poblacion.C5P1"],
                                   'category':["1","1"],
                                   'logical_exp':["equal"],
                                   'operator':['and']}, mensajes=False)

df4=redapyx.frequency(df4)
df4.head(5)

En un aproximado de 15 segundos obtenemos la tabla con la frecuencia de los lugares donde vivian hace 5 años los jefes del hogar que vivie en viviendas alquiladas. Para identificar aquellos que viven fuera de Lima Metropolitana, recodificamos la variable _resp_ diferenciando entre los que no vivian hace 5 años en LM ("no_LM") y los que sí vivian en LM ('si_LM') durante este perido de tiempo.

In [ ]:
#Recodificamos la variable resp en una nueva variable llamada resp2.
df4['resp2']="no_LM"
df4.loc[df4['resp'].str.contains('Continente'), 'resp2'] = "otro_continente"
df4.loc[df4['resp'].str.contains("provincia de Lima"), 'resp2'] = "si_LM"
#agrupamos la variable recodicada por ubigeo sumamdo las fecuencias. luego pivoteamos la BD para tener una matriz de 2x2 con el ubigeo como index
df4=df4.groupby(["ubigeo","resp2"]).sum(numeric_only=True)[['fre']].reset_index().pivot(index="ubigeo",columns="resp2",values="fre")

## sumamos todas las variables para tener el total de los jefes del hogar por lugar donde vivian en los últimos 5 años
df4['jh_total']=df4.sum(axis=1) 
## creamos una variable con el porcentaje de los jefes del hogar que vivian fuera de la provincia de Lima metropolitana
df4['jf_fuera_LM']=((df4.no_LM/df4.jh_total)*100).round(1)
df4['jf_fuera_LM_otro_con']=(((df4.no_LM+df4.otro_continente)/df4.jh_total)*100).round(1)
df4.sort_values("jf_fuera_LM_otro_con", ascending=False).head(6)

Ahora podemos saber que el mayor procentaje de jefes del hogar en viviendas alquiladas que no vivian hace 5 años en la provincia de Lima metropolitan se encuentra en el distrito de Miraflores (150122) con 28% del total de jefes del hogar en el distrito. Llama la atención que de este total, 21% sea de jefes del hogar que vivían en el extranjero. Por su parte, el distrito donde se encuentra el menor número de jefes del hogar que no vivian en Lima Metropolitana hace 5 años es el distrito del Rimac, con solo 9% seguido por el distrito de Chaclacayo con 10% del total de jefes del hogar en vivienda alquiladas.

Ahora que sabemos que cerca del 84% de los jefes del hogar en vivienda alquiladas vivían en la provincia de Lima en los último 5 años, revisemos si estos datos son similares a nivel interdistrital. Para ello usaremos la pregunta sobre el distrito en que vivia el jefe del hogar hace 5 años ("poblacion.C5P6DI"). De forma similar al ejemplo anterior, pasaremos un filtro que considere la variable sobre la tenencia de la vivienda ('vivienda.C2P13') y la relación con el jefe del hogar ('poblacion.C5P1').

In [ ]:
df5=redapyx.query_2017(tipo='Frequency',var1="poblacion.C5P6DI",area_break="distrito",selection="1501",
                      for_query={'variables':["vivienda.C2P13","poblacion.C5P1"],
                                 'category':["1","1"],
                                 'logical_exp':["equal"],
                                 'operator':['and']})

df5=redapyx.frequency(df5)
df5.head()

Procedemos a recodificar la variable resp para separar los lugares de procedencia de acuerdo con el departamento, provincia y distrito. Luego, usaremos la BD espacial de distritos 2017 para agregarle el ubigeo a cada uno de nuestros lugares de procedencia. Una vez recodificado el ubigeo de procedencia ('ubigeo_pro') creamos una nueva variable que identifique el total de jefes del hogar que no vivian en el mismo distrito de LM en los últimos 5 años.

In [ ]:
# Creamos tres nuevas variables con los datos del departamento, provincia y distrito de  procedencia del jefe del hogar. 
df5["resp"]=df5["resp"].str.upper()
df5["dep"]=df5.resp.str.split(",", expand=True)[0]
df5["prov"]=df5.resp.str.split(",", expand=True)[1].str.strip()
df5["dis"]=df5.resp.str.split(":", expand=True)[1].str.strip().replace(["Á","É","Í","Ó","Ú"],["A","E","I","O","U"], regex=True)

## agregamos el ubigeo a los lugares de procendecia. Para ello haremos un loop usando los nombres de los ditritos de la BD espacial de distritos 2017- INEI
df5["ubigeo_pro"]=""
for ubigeo, dis_name in dis17["nombdist"].items():
    df5.loc[(df5.dis==dis_name) & (df5.dep=="LIMA") & (df5.prov=="LIMA"),"ubigeo_pro"]= ubigeo

## Mantenemos únicamente las variables que necesitamos
df5=df5[['resp', 'fre','ubigeo_pro', 'ubigeo']].copy()

## Creamos una nueva variable con el resultado de la comparación entre ubigeos.
df5["resp2"]="otro_distrito_fuera_LM"
df5.loc[((df5['ubigeo_pro']==df5['ubigeo']) & (df5.ubigeo_pro!="")) | (df5['resp']==("LIMA, LIMA, DISTRITO NO ESPECIFICADO")) , 'resp2'] = "mismo_distrito_LM"
df5.loc[(df5['ubigeo_pro']!=df5['ubigeo']) & (df5.ubigeo_pro!=""), 'resp2'] = "otro_distrito_LM"
df5.loc[(df5['resp'].str.contains('PAÍS'))|(df5['resp'].str.contains('CONTINENTE'))|(df5['resp']=="9999"), 'resp2'] = "otro_pais"


In [ ]:
## Agrupamos la nueva variable por ubigeo de destino (el ubigeo donde se ubica el jefe del hogar), sumamos los resultados de la nueva variable ('resp2') y pivoteamos la BD
df6=df5.groupby(["ubigeo","resp2"]).sum(numeric_only=True)[['fre']].reset_index().pivot(index="ubigeo",columns="resp2",values="fre")
## sumamos todas las variables para tener el total de población por tipo de tenencia
df6['jh_total']=df6.sum(axis=1) 
## creamos una variable con el porcentaje de la variable vivienda alquilada con respecto al total de viviendas
df6['jh_otro_dis_LM_p']=((df6.otro_distrito_LM/df6.jh_total)*100).round(1)
df6.sort_values("jh_otro_dis_LM_p", ascending=False).head(5)

La mayoria de los jefes del hogar que hace 5 años vivian en un distrito distito al distrito donde fueron censados se encuentra en el distrito de San Borja y Magdalena con 30.2% y 29.9%, respectivamente. Podemos observar que a diferencia de lo que sucede con el distrito de Santa Rosa (150139), en estos dos el porcentaje de jefes del hogar en viviendas alquiladas que vivian fuera de Lima Metropolitana es de alrededor de 15% lo que los situa por debajo de la media de los distritos de LM (17%).

Estos datos pueden apreciarse mejor si son visualizados en espacialmente. Para ello, volvemos a usar la data spacial de distritos 2017 del INEI. Se revela cierta concentración en los distritos del sur de Lima de hogares que no vivian en la provincia de Lima hace 5 años. Mientras que cambio interdistrital de los hogares que viven en viviendas alquiladas al interior de LM se estaría concentrando en los distritos de Lima centro como San Borja, Magdalena, Puedo Libre, San Miguel y Barranco.

In [ ]:
df7=df4[['jf_fuera_LM_otro_con']].join(df6[['jh_otro_dis_LM_p']].sort_index())
vis2=dis17.merge(df7, right_index=True, left_index=True, how='inner', validate='1:1')

## Usaremos la libreía Matplotlib para graficar el resultado
fig,ax=plt.subplots(1,2, figsize=(12,9))

vis2.plot("jf_fuera_LM_otro_con",ax=ax[0], legend=True, cmap="PuBu")
vis2.plot("jh_otro_dis_LM_p",ax=ax[1], legend=True, cmap="BuGn")

ax[0].set_title("Jefes del hogar en viviendas alquiladas\n que hace 5 años vivian fuera de\n Lima Metropolitana %") 
ax[1].set_title("Jefes del hogar en viviendas alquiladas\n que hace 5 años vivian en otro distrito\n de Lima Metropolitana %")

### Análisis de las redes de procedencia y destino de los hogares que alquilan una vivienda en el 2017

Finalmente, revisamos de forma general lo 5 principales nodos de atracción de hogares que alquilan viviendas en el 2017. Para ello, haremos uso de la libreria _networkx_ la cual permite convertir la Base de datos en un estructura de redes. Para una mejor visualización, solo se considera para el análisis los 4 primeros distritos de procedencia para cada uno de nuestros 5 principales destinos seleccionados.

In [ ]:
# libraries
import numpy as np
import networkx as nx

def keep_first2(df):
    df=df.sort_values('fre', ascending=False).iloc[0:4]
    return df

df8=(df5.
     groupby(["ubigeo","ubigeo_pro","resp2"]).
     sum(numeric_only=True)[['fre']].
     reset_index('resp2').
     query('resp2=="otro_distrito_LM"').
     groupby(level=0).
     apply(keep_first2).
     reset_index(level=0, drop=True).
     reset_index()
    )

v1=df8.query('ubigeo=="150130" | ubigeo=="150120" | ubigeo=="150139" | ubigeo=="150138"|ubigeo=="150131"').copy()
v2=df8.query('ubigeo=="150130" |ubigeo=="150131"').copy()

# # Build your graph
G1=nx.from_pandas_edgelist(v1, 'ubigeo_pro', 'ubigeo', create_using=nx.DiGraph())
G2=nx.from_pandas_edgelist(v2, 'ubigeo_pro', 'ubigeo', create_using=nx.DiGraph())

# Plot it
f, ax=plt.subplots(1,2, figsize=(12,9))
nx.draw(G1, with_labels=True, arrows=True, node_size=250, font_size=8, width=2.7,node_color='skyblue', alpha=0.8, edge_color=v1['fre'],ax=ax[0],edge_cmap=plt.cm.copper_r)
nx.draw(G2, with_labels=True, arrows=True, node_size=250, font_size=8, width=2.7,node_color='skyblue', alpha=0.8, edge_color=v2['fre'],ax=ax[1],edge_cmap=plt.cm.copper_r)

plt.show()

Del gráfico anterior se puede observar que los 3 de los 5 distritos de mayor cantidad de hogares en viviendas alquiladas se consolidan como claros nodos de atracción. A su vez, 4 cuatro de los 5 distritos son lugares de recepción de hogares que hace 5 años vivian en viviendas alquiladas en el distrito de Miraflores, donde destaca el ditrito de San Isidro, Santa Maria del Mar y Magdalena del Mar. Finalmente, llama la atención la dinámica entre San Borja (150130), San Isidro (150131), Santiago de Surco (150140) y La Molina (150114), cuyos distritos forman un subsistema de nodos y flujos.